<a href="https://colab.research.google.com/github/lmuther8/Predictive-Machine-Failure/blob/main/1_cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using ML to predict machinery failure

This notebook takes all the orignal csvs and systmeatically combines them into one master DataFrame for statistical processing in a nother notebook.

Incorporates telemetry data combined with errors, failures and number of days since a component has had maintenance on it.

##Packages and Configs

In [ ]:
#Connecting to G Drive
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/ColabNotebooks/PredictiveModelling/data/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/ColabNotebooks/PredictiveModelling/data


In [ ]:
path = %pwd

In [ ]:
import pandas as pd
import numpy as np

##Read-in/augment data and create dataframes

In [ ]:
#read CSVs and change datetime string to datetime object

telemetry_df = pd.read_csv('telemetry.csv')
telemetry_df['datetime'] = pd.to_datetime(telemetry_df['datetime'], format='%m/%d/%Y %I:%M:%S %p')

errors_df = pd.read_csv('errors.csv')
errors_df['datetime'] = pd.to_datetime(errors_df['datetime'], format='%m/%d/%Y %I:%M:%S %p')

maint_df = pd.read_csv('maint.csv')
maint_df['datetime'] = pd.to_datetime(maint_df['datetime'], format='%m/%d/%Y %I:%M:%S %p')

failures_df = pd.read_csv('failures.csv')
failures_df['datetime'] = pd.to_datetime(failures_df['datetime'], format='%m/%d/%Y %I:%M:%S %p')

#do not think I will ever use this data
#machines_df = pd.read_csv('machines.csv')

Reformat errors and maintenance

In [ ]:
errors_count_df = pd.get_dummies(errors_df.set_index('datetime')).reset_index()
errors_count_df.columns = ['datetime', 'machineID', 'error1', 'error2', 'error3', 'error4', 'error5']
errors_count_df = errors_count_df.groupby(['machineID', 'datetime']).sum().reset_index()

In [ ]:
print(errors_df.shape)
errors_count_df.shape

(3919, 3)


(3616, 7)

In [ ]:
maint_count_df = pd.get_dummies(maint_df.set_index('datetime')).reset_index()
maint_count_df.columns = ['datetime', 'machineID', 'comp1', 'comp2', 'comp3', 'comp4']
maint_count_df = maint_count_df.groupby(['machineID', 'datetime']).sum().reset_index()

maint_count_df = telemetry_df[['datetime', 'machineID']].merge(maint_count_df, how='left', on=['datetime','machineID']).fillna(0)

In [ ]:
print(maint_df.shape)
maint_count_df.shape

(3286, 3)


(876100, 6)

In [ ]:
components = ['comp1', 'comp2', 'comp3', 'comp4']
machineID = 0
maint_days_since = []
prev_list = []

for ind, row in maint_count_df.iterrows():
  new_list = [row['datetime'],row['machineID']]
  if row['machineID'] == machineID:
    for i, c in enumerate(components):
      if row[c]==0:
        new_list.append(prev_list[i+2]+(1/24))
      else:
        new_list.append(0)
    prev_list = new_list.copy()
  else:
    new_list.extend([row['comp1'], row['comp2'], row['comp3'], row['comp4']])
    machineID = row['machineID']
    prev_list = new_list.copy()
  maint_days_since.append(new_list)



In [ ]:
maint_days_since_df = pd.DataFrame(maint_days_since, columns=['datetime', 'machineID','comp1', 'comp2', 'comp3', 'comp4'])

In [ ]:
maint_days_since_df[50:55]

,datetime,machineID,comp1,comp2,comp3,comp4
50,2015-01-03 08:00:00,1,2.083333,2.083333,2.083333,2.083333
51,2015-01-03 09:00:00,1,2.125000,2.125000,2.125000,2.125000
52,2015-01-03 10:00:00,1,2.166667,2.166667,2.166667,2.166667
53,2015-01-03 11:00:00,1,2.208333,2.208333,2.208333,2.208333
54,2015-01-03 12:00:00,1,2.250000,2.250000,2.250000,2.250000


Combine all DataFrames into one: `master_df`



In [ ]:
master_df = telemetry_df.merge(maint_days_since_df, how='left', on=['datetime','machineID'])
master_df = master_df.merge(errors_count_df, how='left', on=['datetime','machineID'])
master_df = master_df.merge(failures_df, how='left', on=['datetime','machineID'])

In [ ]:
master_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,comp1,comp2,comp3,comp4,error1,error2,error3,error4,error5,failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.041667,0.041667,0.041667,0.041667,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0.083333,0.083333,0.083333,0.083333,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0.125000,0.125000,0.125000,0.125000,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0.166667,0.166667,0.166667,0.166667,NaN,NaN,NaN,NaN,NaN,NaN


###Output master df to drive as csv

In [ ]:
with open(path+'/combined_data.csv', 'w', encoding = 'utf-8-sig') as f:
  master_df.to_csv(f)